<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/12estadisticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero

### Estadísticas básicas
Vamos a analizar este tipo de variables

### Índice
[Centralidad](#Centralidad)<br>
[Dispersión](#Dispersión)<br>
[Histogramas](#Histogramas)<br>
[Asimetría](#Asimetría)<br>
[Curtosis](#Curtosis)<br>
[Diagramas de Barras](#Barras)<br>
[Tests estadísticos](#Tests)<br>


<a name="Centralidad"></a>
## Centralidad

La idea es intentar reducir la variable completa a un solo valor, un "centro". Dos valores principales

* Media $\mu(x) = \frac{\displaystyle {\sum_{i=1}^{N} x_i}}{N}$, donde $x$ es la variable que estamos estudiando formada por $x_1, \dots, x_N$. La media es la medida de centralidad más popular. Puede verse afectada si hay hay una proporción grande de valores demasiado grandes o pequeños (outliers)

* Mediana: valor que deja al 50% de los valores por debajo y el otro 50% por encima



Ejemplo: notas obtenidas por diferentes países en las pruebas Pisa en lectura (REA), matemáticas (MAT) y ciencias (SCI) tanto para mujeres (FE) como para hombres (MA). Incluye también la renta per capita (RPC) del país y el nombre (PAIS) del país.

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/PisaDataClean.csv"
df_pisa = pd.read_csv(url)
df_pisa

Una forma de ver una dataframe de datos numéricos resaltando los valores extremos:

In [ ]:
df_pisa.style.background_gradient("RdYlGn")

In [ ]:
df_pisa.info() # información general, nulos, tipos y memoria que ocupa

In [ ]:

desc_pisa = df_pisa.describe()
desc_pisa

In [ ]:
# Nos quedamos solo con media y mediana
mediamediana = desc_pisa.loc[["mean","50%"]]
mediamediana

In [ ]:
# veamos la diferecia
mediamediana.iloc[0,:]-mediamediana.iloc[1,:]

In [ ]:
# otra forma, solo para una variable
df_pisa.MAT.mean(), df_pisa.MAT.median()

<a name="Dispersión"></a>
## Dispersión

Si las medidas de centralidad dan la idea de un "centro" de la variable, la media de dispersión sería el "radio" indica lo alejados que están de ese centro. Vamos a ver 2 cada uno relacionado con una de las medidas de centralidad

* Desviación típica $\sigma(x)=\sqrt{\frac{{\displaystyle \sum_{i=1}^{N}\left(x_{i}-\mu\right)^{2}}}{N}}$, la raíz cuadrada de la varianza

* Desviación absoluta con respecto a la mediana $\mathit{MAD}(x) = mediana(|x_i - mediana(x)|)$

In [ ]:
df_pisa.MAT.std(), (df_pisa.MAT - df_pisa.MAT.median()).abs().median()

In [ ]:
# nos quedamos solo con los datos que sean números (asumimos que hemos comprobado que todos son ratio o intervalo)
df_pisa_num = df_pisa.select_dtypes(include=["number"])
datos = []
for c in df_pisa_num.columns:
    variable = df_pisa[c]
    datos.append([variable.mean(), variable.median(), variable.std(), (variable-variable.median()).abs().median()])

estad = pd.DataFrame(datos,columns=["mean","median","std","MAD"],index=df_pisa_num.columns)
estad

Algunas consecuencias sencillas:
* De media parece que los hombres lo hacen mejor en matemáticas y las mujeres en lectura, en ciencia la diferencia es muy pequeña
* De media se obtiene mejor nota en ciencias que en lectura, y en lectura que en matemáticas
* En general la mediana es mayor que la media indicando una mayor dispersión a la izquierda
* La mayor dispersión `std` se da en MAT_MA, pero si nos fijamos en la mediana es SCI_MA (quizás MAT_MA tiene más outliers?)
* La menor dispersión `std` se da en REA_FE y en SCI_FE, aunque desde el punto de vista de MAD se da en SCI_FE y MAT_FE. En todo caso parece que las notas para las chicas varían menos de país en país que en el caso de los chicos (¿por qué?)

<a name="Histogramas"></a>
## Histogramas

* No debe confundirse con diagrama de barras, donde se representan datos categóricos nominal

* Un histograma representa la frecuencia (número de elementos) en una variable (ratio u intervalo) representada por intervalos, nos permite ver la distribución de la variable (algo que no se pretende con el diagrama de barras)

* En el caso de valores ordinales se puede usar un "diagrama de barras ordenado"

Ejemplo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



#generate a random numpy array with 1000 elements
normaldata = np.random.randn(100000)
data=normaldata

#histograma
plt.hist(data,edgecolor="black", bins =30)

#añadimos título
plt.title("Histograma")

#etiqueta en X
plt.xlabel("Valores")

#etiqueta en y
plt.ylabel("Frecuencias")

# mostrarlo
plt.show()

In [ ]:
data.mean(), data.std(), np.median(data)

Con `density=True` conseguimos frecuencias relativas

In [ ]:
#histograma
plt.hist(data,edgecolor="black", bins =30, density=True)

#añadimos título
plt.title("Histograma")

#etiqueta en X
plt.xlabel("Valores")

#etiqueta en y
plt.ylabel("Frecuencias")

# mostrarlo
plt.show()

Cuando tenemos una normal "perfecta"  se tiene que
<img src="https://news.mit.edu/sites/default/files/styles/news_article__image_gallery/public/images/201202/20120208160239-1_0.jpg?itok=1X1a_HCs" width=400></img>

13.6 + 34.1 + 13.6 + 34.1 = 95.4% es decir casi el 95% de los valores estarán entre la media y +- 2 la desviación típica

In [ ]:
m = data.mean()
s = data.std()

sum((data < m+2*s) & (data >m-2*s))/len(data)

<a name="Asimetría"></a>
## Asimetría

Asimetría a la derecha: muchos datos acumulados en poco espacio a la izquierda, a la derecha un descenso lento y prolongado

* Se tendrá que media>mediana
* Normalmente solo tendremos que preocuparnos por outliers a la derecha, es decir por valores "excesivamente grandes"
* Sucede por ejemplo en mediciones que por su naturaleza son todas positivas
* Puede tener sentido hacer un estudio diferente a la izquierda y a la derecha de la mediana

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/madrid/contaminacionLargo.csv"
df_conta = pd.read_csv(url)
df_conta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

data = df_conta.PM10
#histograma
plt.hist(data,edgecolor="black", bins =50)

#añadimos título
plt.title("Histograma")

#etiqueta en X
plt.xlabel("Valores")

#etiqueta en y
plt.ylabel("Frecuencias")

# mostrarlo
plt.show()

In [ ]:
data.mean(), data.median()

Como vemos aquí no se cumple la regla anterior de que en el entorno 2std se concentra el 95% de la población. Esto es así porque no se trata de una normal

In [ ]:
m = data.mean()
s = data.std()

sum((data < m+2*s) & (data >m-2*s))/len(data)

La función `skew` de Pandas nos indica la asimetría:

        >0 : Asimetría a la derecha o positiva
        aprox. 0 : simétrico
        <0 : asimetría a la izquierda o negativa


<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Relationship_between_mean_and_median_under_different_skewness.png"  width=500>By Diva Jain - https://codeburst.io/2-important-statistics-terms-you-need-to-know-in-data-science-skewness-and-kurtosis-388fef94eeaa, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=84219892</img>





In [ ]:
data.skew()

Tenemos por tanto asimetría a la derecha. En el dataframe de PISA

In [ ]:
for c in df_pisa_num:
    print(c,df_pisa_num[c].skew())

En nuestros datos "normales"

In [ ]:
pd.DataFrame({"x": normaldata}).skew()

Como vemos no sale exactamente 0 aunque sí muy cercano

<a name="Apuntamiento"></a>
## Curtosis

La curtosis indica el peso de las colas en relación con una normar estándar. A menudo se confunde con "apuntamiento" pero no es exactamente lo mismo. La función `kurtosis` de Pandas nos indica este valor:

        >0 : leptocúrtica ; los outliers tienen más peso que en la normal, tenemos muchos outliers (hay que ver por qué y si merece la pena hacer un estudio solo de esta parte)
        aprox. 0 : ismilar a una normal
        <0 : los outliers tienen menos peso que en la normal, la distribución está más concentrada alrededor de la media



In [ ]:
pd.DataFrame({"x": normaldata}).kurtosis()

In [ ]:
df_conta.PM10.kurtosis()

In [ ]:
df_pisa.MAT.kurtosis()

## Ejemplo

Datos de 7 sensores de radiación solar durante varios días con todas sus horas

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/raw/master/datos/solar.zip"
df_solar = pd.read_csv(url)

In [ ]:
df_solar

In [ ]:

import matplotlib.pyplot as plt
for c in df_solar.columns[4:]:
    fig, ax = plt.subplots(figsize=(24, 6))
    df_solar[c].hist(bins=50)
    plt.title(c)
    plt.show()

¿Qué consecuencias se extraen a simple vista? Añadir código para probar

<a name="Barras"></a>
## Diagramas de barras y de tarta

In [ ]:
import pandas as pd
import seaborn as sns
url= "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pokemon.csv"

# solución
df_pok = pd.read_csv(url)

df_pok

In [ ]:

sns.countplot(data=df_pok, x="Generation")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

frecuencias =  df_pok.Generation.value_counts()
labels = frecuencias.index
sizes = frecuencias.values

plt.pie(sizes, labels=labels)
plt.show()

<a name="Múltiples"></a>
## Múltiples histogramas

En ocasiones para comparar interesa reproducir varios histogramas en uno solo. Aunque se puede hacer esto con matplotlib la biblioteca seaborn ofrece un catálogo más amplio de posibilidades

In [ ]:
import seaborn as sns # !pip install seaborn

In [ ]:

sns.histplot(data=df_conta, x="PM10", hue="ANO",palette='tab10')

a menudo se ve mejor si usamos la opción `multiple='stack'`

In [ ]:
sns.histplot(data=df_conta, x="PM10", hue="ANO",palette='tab10', multiple="stack")

También se pueden comparar dos variables continuas, pero puede que haya que hacer una transformación

In [ ]:
df_conta_largo =df_conta[["PM10","PM2.5"]].melt()
df_conta_largo

In [ ]:
sns.histplot(data=df_conta_largo, x="value", hue="variable",palette='tab10', multiple="stack")

Incluso 2 variables continuas

In [ ]:
sns.histplot(data=df_conta, x="PM10", y="PM2.5")

<a name="Tests"></a>
## Test Estadísticos



En ciencia de datos no hay casi nunca certezas; al fin y al cabo tenemos unos datos particulares, una "muestra", y por eso algunas conclusiones que saquemos pueden ser meras coincidencias o casualidades. Los tests estadísticos nos indican si una hipótesis se verifica o no con una alta probabilidad.

Cada test va orientado a un problema concreto y plantea una hipótesis,
 llamada *hipótesis nula*.

El resultados del test suele ser un valor p que indica la probabilidad de que se cumpla la hipótesis nula además de otros valores (a menudo un estadístico que depende del test)

Si p<0.05 o p<0.01 (depende del nivel de exigencia) rechazamos la hipótesis nula y damos por válida la contraria, la hipótesis alternativa

Es importante observar que si p>0.05 no "aceptamos" la hipótesis nula, solo decimos que no hemos podido rechazarla (en la práctica a menudo esto se toma como prueba de aceptación aunque no sea correcto).

#### Ejemplo: test de normalidad

In [ ]:
from scipy.stats import normaltest
from scipy import stats

def normal(data):
    _, p = stats.normaltest(data,nan_policy="omit")
    if p<0.05:
        msg = "Se rechaza H0: no sigue una distribución normal"
    else:
        msg = "No se rechaza H0; no podemos descartar  una distribución normal"
    return msg,round(p,4)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#generate a random numpy array with 100000 elements
normaldata = np.random.randn(100000)

plt.hist(normaldata,edgecolor="black", bins =30, density=True)

normal(normaldata)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normales(df):
    for c in df.columns:
        fig, ax = plt.subplots(figsize=(5, 3))
        df[c].hist(bins=20)
        msg,p = normal(df[c])
        plt.title(f"{c} {msg} - p: {p} ")
        plt.show()

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/PisaDataClean.csv"
df_pisa = pd.read_csv(url)
# nos quedamos solo con los datos que sean números (asumimos que hemos comprobado que todos son ratio o intervalo)
df_pisa_num = df_pisa.select_dtypes(include=["number"])
normales(df_pisa_num)

#### Ejemplo: test de igualdad de la media

Supongamos que tenemos 2 monedas, en la primera nos salen 2 caras seguidas, en la segunda 2 cruces seguidas ¿podemos asegurar que son monedas diferentes? Casi todos estaremos de acuerdo en que no, en que la diferencia se puede deber al azar. Eso sí, si tiramos cada moneda 1000 veces y en la primera sale por ejemplo un 30% más de caras que en la segunda puede que sospechemos que sí lo sean. Estas ideas son las que implementan los tests que comparan las medias de dos variables. Vamos a ver 2:

t de student: solo aplicable bajo ciertas circunstancias (es un test paramétrico): normalidad de ambas variables, misma varianza...la hipótesis nula (que rechazaremos si p<0.05) es que la media es la misma

Kolmogorov-Sminorv: se puede aplicar a cualquier pareja de variables (pero es más exigente). La hipótesis nula (que rechazaremos si p<0.05) es que los datos provienen de la misma distribución continua

*t de Student*



In [ ]:
from scipy.stats import ttest_ind
x1 =  np.random.randn(100000)
x2 = np.random.randn(100000)
ttest_ind(x1, x2)

como pvalue>0,05 no podemos rechazar la hipótesis de que la media es muy similar

In [ ]:
from scipy.stats import ttest_ind
x1 = np.random.randn(100000)+1
x2 = np.random.randn(100000)
ttest_ind(x1, x2)

Podemos rechazar que ambas medias sean iguales.

*Kolmogorov-Smirnov*

Realmente este test no busca comparar medidas sino ver si dos muestras proceden de la misma población. Veamos un ejemplo

In [ ]:
import pandas as pd
url = r"https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/madrid/Cont_Meteo_Traf.csv"
df_data =  pd.read_csv(url,parse_dates=['FECHAH'])
df_data["year"] = df_data.FECHAH.dt.year
df_data.columns

In [ ]:
df_data.year.value_counts()

In [ ]:
years = df_data.year.unique()
years = np.sort(years)
# un array de dataframes, uno por año
df_data_year = [df_data[df_data.year==y]["TEMPERATURA"] for y in years ]

In [ ]:
for y in years[1:]:
    t = df_data[df_data.year==y]["TEMPERATURA"]
    print(y,normal(t))

No son normales, vamos a utilizar el test de kolmogorov-smirnov para ver si la temperatura difiere de forma significativa en distintos años.

In [ ]:
from scipy.stats import kstest

for y1 in years[1:]:
    for y2 in years[1:]:
        if y2>y1:
            t1 = df_data[df_data.year==y1]["TEMPERATURA"]
            t2 = df_data[df_data.year==y2]["TEMPERATURA"]
            print(y1,y2,kstest(t1, t2),round(t1.mean(),2),round(t2.mean(),2))


Todos los años corresponden a temperaturas diferentes con una diferencia estadísticamente signficativa